In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

--2023-01-04 20:18:11--  https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz.1’

spark-3.3.1-bin-had 100%[===================>] 285.48M  22.9MB/s    in 13s     

2023-01-04 20:18:25 (21.5 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz.1’ saved [299350810/299350810]



In [4]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [7]:
import findspark
findspark.init()

In [8]:
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [38]:
# Import necessary packages
import glob
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from datetime import date
from pyspark.sql.functions import lit
from datetime import datetime

In [39]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [40]:
spark

In [41]:
files = glob.glob('/content/*.parquet')
list_of_files = [i for i in files if not i.endswith("tr.parquet")]
latest_file = max(list_of_files, key=os.path.getctime)
#print(latest_file)

In [42]:
file_name = latest_file.rsplit('/', 1)[1]
#print(file_name) 

In [43]:
df = spark.read.parquet(latest_file)

In [44]:
#df.show()

In [45]:
df2 = df.select(explode("tracks.items"))
#df2.printSchema()

In [46]:
df2 = df2.select("col.added_at","col.added_by","col.is_local","col.primary_color","col.track","col.video_thumbnail")
#df2.show()

In [47]:
#df2.printSchema()

In [48]:
df3 = df2.select("track.album.name", "track.duration_ms", "track.episode", "track.popularity", 
           "track.type", "track.uri", "track.album.external_urls.spotify",
           "track.album.album_type", "track.album.release_date", 
           "track.album.total_tracks", "track.album.release_date_precision",
           "track.album.type", "track.artists.name", "track.artists.id",
           "track.artists.href", "track.artists.type", "track.artists.uri")

In [49]:
#df3.show()

In [50]:
df4 = df3.withColumn(
    "ranking",
    row_number().over(Window.orderBy(monotonically_increasing_id()))
)

In [51]:
#df4.show(51)

In [52]:
df4 = df4.withColumn("date", lit(str(date.today())))
df5 = df4.withColumn("time", lit(datetime.now().strftime("%H:%M:%S")))
#df5.show()

In [53]:
newColumns = ["title","duration_ms","episode","popularity",
              "song_type","song_uri","spotify","album_type","release_date","total_tracks","release_date_precision",
              "type", "artists_name", "id", "href", "artists_types", "artists_uri", "ranking", "date", "time"]
df5 = df5.toDF(*newColumns)

In [54]:
df5.write.parquet(file_name[:-8]+"tr"+file_name[-8:]) 

AnalysisException: ignored